In [4]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from scipy.optimize import minimize

In [5]:
data = fpa_data = pd.read_stata('chile.dta')

In [8]:
olsdata = data.copy()
olsdata[['routput','totlab','rcapstock']] = np.log(olsdata[['routput','totlab','rcapstock']])

olsmodel = smf.ols('routput ~totlab + rcapstock',olsdata)
olsresult = olsmodel.fit()
olsresult.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                routput   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                 3.452e+04
Date:                Wed, 08 May 2019   Prob (F-statistic):               0.00
Time:                        08:35:58   Log-Likelihood:                -23690.
No. Observations:               20730   AIC:                         4.739e+04
Df Residuals:                   20727   BIC:                         4.741e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.1804      0.024    214.199      0.000       5.133       5.228
totlab         0.8194      0.008    102.168      0.000       0.804       0.835
rcapstock      0.3317      0.004     91.134      0.000       0.325       0.339
==============================================================================
Omnibus:                     1840.946   Durbin-Watson:                   0.531
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3645.736
Skew:                           0.597   Prob(JB):                         0.00
Kurtosis:                       4.672   Cond. No.                         47.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
fedata = olsdata.copy()

for var in ['routput','totlab','rcapstock']:
    fedata[var+'fe'] = fedata[var]
    for fe in ['id']:
        dum = pd.get_dummies(fedata[fe])
        means = np.tile(fedata[var],len(dum.columns)).reshape(fedata.shape[0],len(dum.columns))
        means = np.array(means*dum)
        fedata[var+'fe'] = fedata[var+'fe'] - means.sum(axis=1)/np.array(dum.sum(axis=1))
       
        
print fedata.columns

Index([u'ciiu_3d', u'id', u'year', u'routput', u'totlab', u'renerg',
       u'realmats', u'rcapstock', u'rinvest', u'routputfe', u'totlabfe',
       u'rcapstockfe'],
      dtype='object')


In [48]:
femodel = smf.ols('routputfe ~totlabfe + rcapstockfe',fedata)
feresult = femodel.fit()
feresult.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              routputfe   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                 3.445e+04
Date:                Wed, 08 May 2019   Prob (F-statistic):               0.00
Time:                        08:55:28   Log-Likelihood:                -30908.
No. Observations:               20730   AIC:                         6.182e+04
Df Residuals:                   20727   BIC:                         6.185e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0069      0.007      0.923      0.356      -0.008       0.022
totlabfe        0.8197      0.008    101.653      0.000       0.804       0.835
rcapstockfe     0.3329      0.004     91.215      0.000       0.326       0.340
==============================================================================
Omnibus:                      361.177   Durbin-Watson:                   1.273
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              739.416
Skew:                          -0.019   Prob(JB):                    2.74e-161
Kurtosis:                       3.924   Cond. No.                         3.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
femodel = smf.mixedlm('routput ~ totlab +rcapstock', olsdata, groups=data['id'])
feresult = femodel.fit()
print feresult.summary()

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: routput    
No. Observations: 20730   Method:             REML       
No. Groups:       2673    Scale:              0.1335     
Min. group size:  1       Likelihood:         -12895.3255
Max. group size:  18      Converged:          Yes        
Mean group size:  7.8                                    
----------------------------------------------------------
           Coef.  Std.Err.     z     P>|z|  [0.025  0.975]
----------------------------------------------------------
Intercept  7.116     0.045  157.791  0.000   7.027   7.204
totlab     0.616     0.007   85.294  0.000   0.602   0.630
rcapstock  0.194     0.004   43.859  0.000   0.186   0.203
Group Var  0.641     0.059                                

